In [1]:
import os
import pandas as pd

os.chdir("D:/Drive/FHNW/zaRepos/fhnw_ds_fs2021_medical_challenge")
rel_path = "./data/uveitis_data.xlsx"

#function that reads data
def read_data(rel_path, **kwargs):
    """
    Generalized function that reads a xlsx, csv, json or html data structure into a pandas dataframe
    :param rel_path: relative path to data to be read
    :param kwargs:  parameters for pd.read_ functions
    """
    filename, file_extension = os.path.splitext(rel_path)
    print(os.path.basename(rel_path) + " will be read")
    if file_extension == ".xlsx": #when reading excel a usefull kwargs will be na_values = "dict of values to consider na"
        df = pd.read_excel(rel_path, **kwargs)
        print("reading filetype "+file_extension)
    elif file_extension == ".csv":
        df = pd.read_csv(rel_path, **kwargs)
        print("reading filetype "+file_extension)
    elif file_extension == ".json":
        df = pd.read_json(rel_path, **kwargs)
        print("reading filetype "+file_extension)
    elif file_extension == ".html":
        df = pd.read_html(rel_path, **kwargs)
        print("reading filetype "+file_extension)
    else:
        print("Filetype not supported by function")
        df = 0

    if df.size != 0:
        print("successfully created table with ", df.size, "values and loaded as df")
        print("the table is ",df.shape[1], "wide and ",df.shape[0],"long")

    return df

df1 = read_data(rel_path)

uveitis_data.xlsx will be read
reading filetype .xlsx
successfully created table with  126850 values and loaded as df
the table is  118 wide and  1075 long


In [2]:
#function to return list of columns to convert to which data type (with choice) according to given list
col_name_data_type = "data_type"
col_index= "old col name"
list_path = "./data/col_names&data_type-Copy1.xlsx"
data_type = "numerical"

def list_of_totype(list_path, col_index, col_name_data_type, data_type):
    """
    :param list_path: path of excel document to read data type from
    :param col_index: index column to use (column name of data type)
    :param col_name_data_type: name of column where data type is stored
    :param data_type: type of data desired. Should be numerical, categorical, char or both (appelation for edge cases)
    :return: returns a list of column names with the desired data type
    """
    col = pd.read_excel(list_path, index_col = col_index)[col_name_data_type] #get content of column with name data_type
    col_to_type = col.where(col == data_type).dropna().index.tolist()
    return col_to_type

#item_filter = list_of_totype(list_path, col_index, col_name_data_type, data_type)
#print(item_filter)

In [38]:
#function that should return all values that couldn't be correctly coerced by next function (to be used in another function)
desired_dtype = ["Int64", "float64"]

def problem_columns(matches, desired_dtype):
    """
    :param matches: feeded from another function
    :param desired_dtype: a list or string that contains the desired data_types that were transformed correctly
    :return: returns a dataframe with all the problematic columns
    """
    for key, value in matches.iteritems():
        if value.dtype in desired_dtype:
            print("\n", key, "is desired", value.dtype, "and will be popped from problematic list \n")
            matches.pop(key)
        else:
            print(key, "has unwanted dtype, keeping for transformation")
    return matches

In [39]:
#function to coerce columns to desired datatype
def coerce_then_problems(dframe, list_path, col_index, col_name_data_type, data_type, desired_dtype):
    """
    :param dframe: dataframe to be checked and wrecked
    :param list_path: path of excel document to read data type from
    :param col_index: index column to use (column name of data type)
    :param col_name_data_type: name of column where data type is stored
    :param data_type: type of data desired. Should be numerical, categorical, char or both (appelation for edge cases)
    :param desired_dtype: a list or string that contains the desired data_types that were transformed correctly
    :return: a dataframe with values that couldn't be coerced to the desired data_type automagically via convert_dtypes()
    """

    item_filter = list_of_totype(list_path, col_index, col_name_data_type, data_type)
    matches = dframe.filter(items = item_filter).convert_dtypes()
    print(matches)
    matches = problem_columns(matches, desired_dtype)
    return matches

In [40]:
coerce_then_problems(df1, list_path, col_index, col_name_data_type, data_type, desired_dtype)

#function that iterates trough columns given by previous function to coerce values that aren't numerical in nature to something else (0, , NaN, NA, 99)






      SUBJECT ID Calcium (Blood) Lactate Dehydrogenase (LDH) (Blood)  \
0              1            2.27                                 184   
1              2            2.32                                 209   
2              3            2.44                                 194   
3              4            2.45                                 156   
4              5            2.28                                 175   
...          ...             ...                                 ...   
1070        1071            2.26                                 194   
1071        1072            2.36                                 239   
1072        1073            2.26                                 196   
1073        1074            2.41                                 173   
1074        1075            2.39                                 214   

     C-Reactive Protein, Normal and High Sensitivity (Blood) WBC (Blood)  \
0                                                  1.83    

C:\Users\Lupo_Remastered\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  # This is added back by InteractiveShellApp.init_path()


,Calcium (Blood),Lactate Dehydrogenase (LDH) (Blood),"C-Reactive Protein, Normal and High Sensitivity (Blood)",WBC (Blood),RBC (Blood),Hemoglobin (Blood),Hematocrit (Blood),MCV (Blood),MCH (Blood),MCHC (Blood),RDW (Blood),Platelet Count (Blood),Neutrophil % (Blood),Lymphocytes % (Blood),Angiotensin Conv#Enzyme (Blood),"Lysozyme, Plasma (Blood)",Anti-Dnase B (Blood),Complement C4 (Blood),Rheumatoid Factor (Blood)
0,2.27,184,1.83,6.91,4.97,14.6,41.4,83.3,29.4,35.3,13.9,194,64.4,30.8,NaN,3,NaN,33,<15
1,2.32,209,<0.15,4.24,4.22,11,36,85.3,26.1,30.6,14.8,260,40.1,49.1,27.3,NaN,NaN,22.8,<10
2,2.44,194,0.137,6.38,4.31,13.6,38.7,89.8,31.6,35.1,11.7,344,70.2,22.1,60.2,4.8,NaN,18.2,<10
3,2.45,156,3.5,5.24,4.82,14.6,43.6,90.5,30.3,33.5,12.4,220,56.6,34.2,38.9,:,NaN,19.9,<15
4,2.28,175,1,8.31,4.72,14,40.7,86.2,29.7,34.4,12.1,324,60.4,27.4,18.3,4.5,NaN,19,<10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,2.26,194,15.4,7.22,5.49,12.7,40.6,74,23.1,31.3,15.4,292,67.3,17.7,< 1.0,9.3,NaN,36.1,<15
1071,2.36,239,0.8,7.65,4.56,12.6,35.9,78.7,27.6,35.1,13.5,219,54.3,38.6,53,3.6,NaN,22.3,<15
1072,2.26,196,0.6,3.83,4.54,14.5,42.8,94.3,31.9,33.9,14.9,239,52.6,35.5,39.9,2.9,NaN,24.6,<15
1073,2.41,173,6.97,16.65,4.94,13.4,40.5,82,27.1,33.1,15.7,369,70.5,24.3,23,3.6,NaN,26,<15
